In [2]:
import nest_asyncio
nest_asyncio.apply()
from pathlib import Path

import sys
sys.path.append("../")

from desci_sense.shared_functions.init import init_multi_chain_parser_config
from desci_sense.shared_functions.parsers.multi_chain_parser import MultiChainParser
from desci_sense.evaluation.utils import get_dataset

from desci_sense.shared_functions.dataloaders import (
    scrape_post,
    convert_text_to_ref_post,
)

In [5]:
config = init_multi_chain_parser_config(llm_type="google/gemma-7b-it",
                                        post_process_type="combined")

In [6]:
multi_chain_parser = MultiChainParser(config)

2024-05-02 10:38:30.706 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:66 - Initializing MultiChainParser. PostProcessType=combined
2024-05-02 10:38:30.761 | INFO     | desci_sense.shared_functions.parsers.multi_chain_parser:__init__:73 - Initializing post parsers...
2024-05-02 10:38:30.763 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'refs_tagger' 
2024-05-02 10:38:30.858 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'topics' 
2024-05-02 10:38:30.871 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'keywords' 
2024-05-02 10:38:30.894 | INFO     | desci_sense.shared_functions.parsers.post_parser_chain:__init__:26 - Initializing parser chain 'hashtags' 


In [4]:
# example raw text posts
TEST_POST_TEXT_W_REF = """
I really liked this paper!
https://arxiv.org/abs/2402.04607
"""

TEST_POST_2_TEXT_W_REF = """
I really disagree with this paper!
https://arxiv.org/abs/2402.04607
"""

TEXT_POSTS = [TEST_POST_TEXT_W_REF, TEST_POST_2_TEXT_W_REF]

In [5]:
# convert raw text to RefPost format for input to the parser
inputs = [convert_text_to_ref_post(p) for p in TEXT_POSTS]

In [12]:
# or scrape posts by url
urls = [
        "https://mastodon.social/@psmaldino@qoto.org/111405098400404613",
        "https://mastodon.social/@UlrikeHahn@fediscience.org/111732713776994953",
        "https://mastodon.social/@ronent/111687038322549430",
    ]
posts = [scrape_post(url) for url in urls]

In [7]:
# batch process
results = multi_chain_parser.batch_process_ref_posts(inputs,active_list=['topics','keywords'],batch_size=10)

2024-05-02 10:39:16.297 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:54 - Retry attempt 1
2024-05-02 10:39:16.299 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:76 - target_url=https://medium.com/@charles.adjovu/dgov-score-framework-donation-acknowledgements-d532584372bd
2024-05-02 10:39:16.321 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:54 - Retry attempt 1
2024-05-02 10:39:16.321 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:76 - target_url=https://twitter.com/yanaiela/status/1780274904031732222
2024-05-02 10:39:16.323 | WARNING  | desci_sense.shared_functions.web_extractors.citoid:before_retry:54 - Retry attempt 1
2024-05-02 10:39:16.324 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async_retry:76 - target_url=https://twitter.com/rawsalerts/status/1777017207366520949
2024-05-02 10:39:16.325 | WARNIN

  0%|          | 0/467 [00:00<?, ?it/s]

2024-05-02 10:41:59.031 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:245 - Invoking parallel chain...
2024-05-02 10:45:22.889 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:batch_process_ref_posts:256 - Post processing 467 results...


ValidationError: 1 validation error for CombinedParserOutput
item_types.1
  Input should be a valid string [type=string_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.6/v/string_type

In [6]:
#
url = "https://twitter.com/mbauwens/status/1779543397528740338"
post = scrape_post(url)

In [7]:
# single process
result = multi_chain_parser.process_ref_post(post)

2024-04-30 15:42:45.141 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async:20 - target_url=https://icandecide.org/v-safe-data/
2024-04-30 15:42:45.164 | DEBUG    | desci_sense.shared_functions.web_extractors.citoid:fetch_citation_async:20 - target_url=https://icandecide.org/pfizer-documents/
2024-04-30 15:43:04.309 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:184 - Processing post with parsers: ['refs_tagger', 'topics', 'keywords', 'hashtags']
2024-04-30 15:43:04.310 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:186 - Instantiating prompts...
2024-04-30 15:43:04.311 | DEBUG    | desci_sense.shared_functions.parsers.multi_chain_parser:process_ref_post:191 - Invoking parallel chain...


In [8]:
result.topics

['sustainability',
 'software & hardware',
 'academia',
 'climate',
 'news',
 'health',
 'technology',
 'research']

In [16]:
result.filter_classification

<SciFilterClassfication.RESEARCH: 'research'>

In [18]:
result.reference_tagger

['recommendation', 'reading']

In [19]:
result.topics

['economics', 'research']

In [3]:
import wandb
import pandas as pd

wandb.login()

api = wandb.Api()

#TODO move from testing
run = wandb.init(project="testing", job_type="evaluation")

# get artifact path

dataset_artifact_id = (
        'common-sense-makers/filter_evaluation/labeled_tweets_no_threads:v1'
    )

# set artifact as input artifact
dataset_artifact = run.use_artifact(dataset_artifact_id)

# initialize table path
# add the option to call table_path =  arguments.get('--dataset')

# download path to table
a_path = dataset_artifact.download()
print("The path is",a_path)

# get dataset file name

table_path = Path(f"{a_path}/labeled_data_table_no_threads.table.json")


# return the pd df from the table
#remember to remove the head TODO
df = get_dataset(table_path)

table_path = Path(f"{a_path}/handles_chart.table.json")

df_handles = get_dataset(table_path)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


wandb:   2 of 2 files downloaded.  


The path is /Users/shaharorielkagan/sensemakers/nlp/notebooks/artifacts/labeled_tweets_no_threads:v1


In [4]:
import concurrent.futures
from tqdm import tqdm

def process_text(text):
    return convert_text_to_ref_post(text)

# Assuming df['Text'] is your DataFrame column
with concurrent.futures.ThreadPoolExecutor() as executor:
    inputs = list(tqdm(executor.map(process_text, df['Text']), total=len(df['Text'])))


100%|██████████| 467/467 [00:55<00:00,  8.39it/s]


In [ ]:
# batch process
results = multi_chain_parser.batch_process_ref_posts(inputs,active_list=['topics','keywords'],batch_size=10)